In [148]:
# Import necessary packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from keras.models import Sequential
from keras.layers import Dense
from scipy.stats import zscore
from statsmodels.tsa.stattools import adfuller
from pmdarima import auto_arima
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import tensorflow as tf
from tensorflow.keras.layers import Input, Dropout, Dense, LSTM, TimeDistributed, RepeatVector
from tensorflow.keras.models import Model
from tensorflow.keras import regularizers
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score


In [107]:
df= pd.read_csv('archive (14)/walmart_cleaned.csv')
# Display the first few rows and summary information of the dataset to understand its structure
df.head(), df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 421570 entries, 0 to 421569
Data columns (total 17 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Unnamed: 0    421570 non-null  int64  
 1   Store         421570 non-null  int64  
 2   Date          421570 non-null  object 
 3   IsHoliday     421570 non-null  int64  
 4   Dept          421570 non-null  float64
 5   Weekly_Sales  421570 non-null  float64
 6   Temperature   421570 non-null  float64
 7   Fuel_Price    421570 non-null  float64
 8   MarkDown1     421570 non-null  float64
 9   MarkDown2     421570 non-null  float64
 10  MarkDown3     421570 non-null  float64
 11  MarkDown4     421570 non-null  float64
 12  MarkDown5     421570 non-null  float64
 13  CPI           421570 non-null  float64
 14  Unemployment  421570 non-null  float64
 15  Type          421570 non-null  int64  
 16  Size          421570 non-null  int64  
dtypes: float64(11), int64(5), object(1)
memory usage

(   Unnamed: 0  Store        Date  IsHoliday  Dept  Weekly_Sales  Temperature  \
 0           0      1  2010-02-05          0   1.0      24924.50        42.31   
 1           1      1  2010-02-05          0  26.0      11737.12        42.31   
 2           2      1  2010-02-05          0  17.0      13223.76        42.31   
 3           3      1  2010-02-05          0  45.0         37.44        42.31   
 4           4      1  2010-02-05          0  28.0       1085.29        42.31   
 
    Fuel_Price  MarkDown1  MarkDown2  MarkDown3  MarkDown4  MarkDown5  \
 0       2.572        0.0        0.0        0.0        0.0        0.0   
 1       2.572        0.0        0.0        0.0        0.0        0.0   
 2       2.572        0.0        0.0        0.0        0.0        0.0   
 3       2.572        0.0        0.0        0.0        0.0        0.0   
 4       2.572        0.0        0.0        0.0        0.0        0.0   
 
           CPI  Unemployment  Type    Size  
 0  211.096358         8.10

In [108]:
# Data preprocessing
df.drop(columns=['Unnamed: 0'], inplace=True)
df['Date'] = pd.to_datetime(df['Date'])
df.sort_values('Date', inplace=True)
df.set_index('Date', inplace=True)
df['Type'] = df['Type'].astype('category').cat.codes

In [109]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
import numpy as np
import matplotlib.pyplot as plt

# Feature scaling
scaler = StandardScaler()
numerical_cols = ['Weekly_Sales', 'Temperature', 'Fuel_Price', 'MarkDown1', 'MarkDown2', 'MarkDown3', 
                  'MarkDown4', 'MarkDown5', 'CPI', 'Unemployment', 'Size']
df[numerical_cols] = scaler.fit_transform(df[numerical_cols])

# Split the data
target_variable = 'Weekly_Sales'
split_date = '2012-06-01'
train = df.loc[:split_date]
test = df.loc[split_date:]

# Confirm the shapes of the train and test sets
print(train.shape, test.shape)


(359432, 15) (65081, 15)


### arima

In [110]:
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error

# Define and fit the ARIMA model
arima_model = ARIMA(train[target_variable], order=(5, 1, 0))
arima_fit = arima_model.fit()

# Make predictions
arima_forecast = arima_fit.forecast(steps=len(test))

# Evaluate the ARIMA model
arima_mse = mean_squared_error(test[target_variable], arima_forecast)
arima_rmse = arima_mse ** 0.5
print('ARIMA RMSE:', arima_rmse)


C:\Users\medan\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\medan\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\medan\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\medan\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning: No supported index is ava

ARIMA RMSE: 1.192591025555719


In [111]:
arima_mse

1.422273354236042

In [116]:
arima_mae = mean_absolute_error(test[target_variable], arima_forecast)
arima_mae


0.6942278354240268

In [147]:
arima_r2 = r2_score(test[target_variable], arima_forecast)
arima_r2

-0.5120446447070328

### Sarima

In [71]:
from statsmodels.tsa.statespace.sarimax import SARIMAX

# Define and fit the SARIMA model
sarima_model = SARIMAX(train[target_variable], order=(1, 1, 1), seasonal_order=(1, 1, 1, 12))
sarima_fit = sarima_model.fit()

# Make predictions
sarima_forecast = sarima_fit.forecast(steps=len(test))

# Evaluate the SARIMA model
sarima_mse = mean_squared_error(test[target_variable], sarima_forecast)
sarima_rmse = sarima_mse ** 0.5
print('SARIMA RMSE:', sarima_rmse)


C:\Users\medan\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\medan\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\medan\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
C:\Users\medan\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning: No supported index is available. Prediction results w

SARIMA RMSE: 0.9988771339402445


In [83]:
sarima_mse

0.9977555287086771

In [141]:
sarima_mae = mean_absolute_error(test[target_variable], sarima_forecast)
sarima_mae


0.5975929306148261

In [150]:
sarima_r2 = r2_score(test[target_variable], sarima_forecast)
sarima_r2

-0.06073203116509496

In [152]:
lr_r2 = r2_score(y_test, lr_forecast)
lr_r2

0.08954992471936207

### Linear Regression

In [74]:
from sklearn.linear_model import LinearRegression

# Prepare the features and target variable
X_train = train.drop(columns=[target_variable])
y_train = train[target_variable]
X_test = test.drop(columns=[target_variable])
y_test = test[target_variable]

# Define and fit the Linear Regression model
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)

# Make predictions
lr_forecast = lr_model.predict(X_test)

# Evaluate the Linear Regression model
lr_mse = mean_squared_error(y_test, lr_forecast)
lr_rmse = lr_mse ** 0.5
print('Linear Regression RMSE:', lr_rmse)


Linear Regression RMSE: 0.9254166260687549


In [139]:
lr_mse

0.8563959318044777

In [140]:
lr_mae = mean_absolute_error(test[target_variable], lr_forecast)
lr_mae


0.6511821495913411

### LSTM

In [130]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Prepare the data for LSTM
def create_dataset(X, y, time_steps=1):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        Xs.append(X.iloc[i:(i + time_steps)].values)
        ys.append(y.iloc[i + time_steps])
    return np.array(Xs), np.array(ys)

time_steps = 10
X_train_lstm, y_train_lstm = create_dataset(train, train[target_variable], time_steps)
X_test_lstm, y_test_lstm = create_dataset(test, test[target_variable], time_steps)

# Define the LSTM model
lstm_model = Sequential()
lstm_model.add(LSTM(50, activation='relu', input_shape=(time_steps, X_train_lstm.shape[2])))
lstm_model.add(Dense(1))
lstm_model.compile(optimizer='adam', loss='mse')

# Fit the LSTM model
lstm_model.fit(X_train_lstm, y_train_lstm, epochs=40, batch_size=32, verbose=1)

# Make predictions
lstm_forecast = lstm_model.predict(X_test_lstm)

# Evaluate the LSTM model
lstm_mse = mean_squared_error(y_test_lstm, lstm_forecast)
lstm_rmse = lstm_mse ** 0.5
print('LSTM RMSE:', lstm_rmse)


C:\Users\medan\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/40
11232/11232 ━━━━━━━━━━━━━━━━━━━━ 21s 2ms/step - loss: 1.2870
Epoch 2/40
11232/11232 ━━━━━━━━━━━━━━━━━━━━ 20s 2ms/step - loss: 0.9545
Epoch 3/40
11232/11232 ━━━━━━━━━━━━━━━━━━━━ 20s 2ms/step - loss: 0.9519
Epoch 4/40
11232/11232 ━━━━━━━━━━━━━━━━━━━━ 19s 2ms/step - loss: 0.9447
Epoch 5/40
11232/11232 ━━━━━━━━━━━━━━━━━━━━ 19s 2ms/step - loss: 0.9473
Epoch 6/40
11232/11232 ━━━━━━━━━━━━━━━━━━━━ 19s 2ms/step - loss: 0.9364
Epoch 7/40
11232/11232 ━━━━━━━━━━━━━━━━━━━━ 19s 2ms/step - loss: 0.9295
Epoch 8/40
11232/11232 ━━━━━━━━━━━━━━━━━━━━ 19s 2ms/step - loss: 0.9432
Epoch 9/40
11232/11232 ━━━━━━━━━━━━━━━━━━━━ 19s 2ms/step - loss: 0.9404
Epoch 10/40
11232/11232 ━━━━━━━━━━━━━━━━━━━━ 19s 2ms/step - loss: 0.9494
Epoch 11/40
11232/11232 ━━━━━━━━━━━━━━━━━━━━ 19s 2ms/step - loss: 0.9326
Epoch 12/40
11232/11232 ━━━━━━━━━━━━━━━━━━━━ 20s 2ms/step - loss: 0.9401
Epoch 13/40
11232/11232 ━━━━━━━━━━━━━━━━━━━━ 20s 2ms/step - loss: 0.9399
Epoch 14/40
11232/11232 ━━━━━━━━━━━━━━━━━━━━ 19s 2ms/step - 

In [137]:
lstm_mse

0.8850804946277132

In [138]:
lstm_mae = mean_absolute_error(y_test_lstm, lstm_forecast)
lstm_mae


0.624128554142552

In [154]:
lstm_r2 = r2_score(y_test_lstm, lstm_forecast)
lstm_r2

0.059018098719653134

### GRU

In [131]:
from tensorflow.keras.layers import GRU

# Define the GRU model
gru_model = Sequential()
gru_model.add(GRU(50, activation='relu', input_shape=(time_steps, X_train_lstm.shape[2])))
gru_model.add(Dense(1))
gru_model.compile(optimizer='adam', loss='mse')

# Fit the GRU model
gru_model.fit(X_train_lstm, y_train_lstm, epochs=40, batch_size=32, verbose=1)

# Make predictions
gru_forecast = gru_model.predict(X_test_lstm)

# Evaluate the GRU model
gru_mse = mean_squared_error(y_test_lstm, gru_forecast)
gru_rmse = gru_mse ** 0.5
print('GRU RMSE:', gru_rmse)


Epoch 1/40


C:\Users\medan\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


11232/11232 ━━━━━━━━━━━━━━━━━━━━ 22s 2ms/step - loss: 1.0448
Epoch 2/40
11232/11232 ━━━━━━━━━━━━━━━━━━━━ 21s 2ms/step - loss: 0.9533
Epoch 3/40
11232/11232 ━━━━━━━━━━━━━━━━━━━━ 21s 2ms/step - loss: 0.9444
Epoch 4/40
11232/11232 ━━━━━━━━━━━━━━━━━━━━ 21s 2ms/step - loss: 0.9489
Epoch 5/40
11232/11232 ━━━━━━━━━━━━━━━━━━━━ 21s 2ms/step - loss: 0.9438
Epoch 6/40
11232/11232 ━━━━━━━━━━━━━━━━━━━━ 21s 2ms/step - loss: 0.9507
Epoch 7/40
11232/11232 ━━━━━━━━━━━━━━━━━━━━ 21s 2ms/step - loss: 0.9517
Epoch 8/40
11232/11232 ━━━━━━━━━━━━━━━━━━━━ 21s 2ms/step - loss: 0.9462
Epoch 9/40
11232/11232 ━━━━━━━━━━━━━━━━━━━━ 21s 2ms/step - loss: 0.9517
Epoch 10/40
11232/11232 ━━━━━━━━━━━━━━━━━━━━ 21s 2ms/step - loss: 0.9329
Epoch 11/40
11232/11232 ━━━━━━━━━━━━━━━━━━━━ 21s 2ms/step - loss: 0.9503
Epoch 12/40
11232/11232 ━━━━━━━━━━━━━━━━━━━━ 21s 2ms/step - loss: 0.9397
Epoch 13/40
11232/11232 ━━━━━━━━━━━━━━━━━━━━ 21s 2ms/step - loss: 0.9354
Epoch 14/40
11232/11232 ━━━━━━━━━━━━━━━━━━━━ 21s 2ms/step - loss: 0.930

In [135]:
gru_mse

0.8926347929936924

In [136]:
gru_mae = mean_absolute_error(y_test_lstm, gru_forecast)
gru_mae

0.662803268492518

In [157]:
gru_r2 = r2_score(y_test_lstm, gru_forecast)
gru_r2

0.05098667323642836

### TCN

In [132]:
from tensorflow.keras.layers import Input, Conv1D, Dense, Flatten, Add, Activation
from tensorflow.keras.models import Model

def TCN_block(x, filters, kernel_size, dilation_rate):
    conv1 = Conv1D(filters, kernel_size, dilation_rate=dilation_rate, padding='causal')(x)
    conv1 = Activation('relu')(conv1)
    conv2 = Conv1D(filters, kernel_size, dilation_rate=dilation_rate, padding='causal')(conv1)
    skip = Add()([x, conv2])
    return skip

input_layer = Input(shape=(time_steps, X_train_lstm.shape[2]))
x = TCN_block(input_layer, 15, 3, 1)
x = TCN_block(x, 15, 3, 2)
x = Flatten()(x)
output_layer = Dense(1)(x)

tcn_model = Model(input_layer, output_layer)
tcn_model.compile(optimizer='adam', loss='mse')

# Fit the TCN model
tcn_model.fit(X_train_lstm, y_train_lstm, epochs=40, batch_size=32, verbose=1)

# Make predictions
tcn_forecast = tcn_model.predict(X_test_lstm)

# Evaluate the TCN model
tcn_mse = mean_squared_error(y_test_lstm, tcn_forecast)
tcn_rmse = tcn_mse ** 0.5
print('TCN RMSE:', tcn_rmse)


Epoch 1/40
11232/11232 ━━━━━━━━━━━━━━━━━━━━ 19s 1ms/step - loss: 8.5191
Epoch 2/40
11232/11232 ━━━━━━━━━━━━━━━━━━━━ 16s 1ms/step - loss: 1.0198
Epoch 3/40
11232/11232 ━━━━━━━━━━━━━━━━━━━━ 16s 1ms/step - loss: 0.9823
Epoch 4/40
11232/11232 ━━━━━━━━━━━━━━━━━━━━ 16s 1ms/step - loss: 0.9793
Epoch 5/40
11232/11232 ━━━━━━━━━━━━━━━━━━━━ 16s 1ms/step - loss: 0.9776
Epoch 6/40
11232/11232 ━━━━━━━━━━━━━━━━━━━━ 17s 1ms/step - loss: 0.9637
Epoch 7/40
11232/11232 ━━━━━━━━━━━━━━━━━━━━ 17s 1ms/step - loss: 0.9670
Epoch 8/40
11232/11232 ━━━━━━━━━━━━━━━━━━━━ 16s 1ms/step - loss: 0.9705
Epoch 9/40
11232/11232 ━━━━━━━━━━━━━━━━━━━━ 16s 1ms/step - loss: 0.9675
Epoch 10/40
11232/11232 ━━━━━━━━━━━━━━━━━━━━ 16s 1ms/step - loss: 0.9586
Epoch 11/40
11232/11232 ━━━━━━━━━━━━━━━━━━━━ 16s 1ms/step - loss: 0.9518
Epoch 12/40
11232/11232 ━━━━━━━━━━━━━━━━━━━━ 16s 1ms/step - loss: 0.9517
Epoch 13/40
11232/11232 ━━━━━━━━━━━━━━━━━━━━ 16s 1ms/step - loss: 0.9549
Epoch 14/40
11232/11232 ━━━━━━━━━━━━━━━━━━━━ 16s 1ms/step - 

In [133]:
tcn_mse

0.8939660826844532

In [134]:
tcn_mae = mean_absolute_error(y_test_lstm, tcn_forecast)
tcn_mae

0.6624900125966953

In [160]:
tcn_r2 = r2_score(y_test_lstm, tcn_forecast)
tcn_r2

0.049571299706031025